# Warsztaty Python w Data Science

---
## Inżynieria Cech (Feature Engineering) - część 1 z 3  

- ### Operacje na prostych wartościach
  - #### Binaryzacja
  - #### "*Kubełkowanie*" - Binning 

- ### Wygładzanie Laplace'a

- ### Skalowanie wartości
  - #### Logarytmiczne
  - #### Skalowanie Min-Max 
  - #### *Robust scaling*
  - #### Standaryzacja 

- ### Zmienne kategoryczne
  - #### Indeksacja
  - #### *One-hot encoding*
  - #### Porządek w zmiennych kategorycznych



---

## The features you use influence more than everything else the result. 
## No algorithm alone, to my knowledge, can supplement the information gain given by correct feature engineering.
## <div style="text-align: right">— Luca Massaron Autor, Kaggle master</div>

---

## Coming up with features is difficult, time-consuming, requires expert knowledge.
## "_*Applied machine learning*_" is basically feature engineering.
## <div style="text-align: right">— Andrew Ng</div>

---

# Proste wartości<a id="simple"></a>

## Uzasadnienie dla binaryzacji

In [ ]:
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib.dates as mdates

count_df = pickle.load( open( "data/count.p", "rb" ) )

plt.figure(figsize=(24,12))
plt.style.use("dark_background")

chart = sns.scatterplot(data=count_df)

pickle.dump( count_df, open( "data/count.p", "wb" ) )



---
## "*Kubełkowanie*" - Binning 


### Histogram o stałej szerokości 

In [ ]:
import numpy as np
import pandas as pd

df = pd.DataFrame({'value': np.random.randint(0, 100, 20)})
df

In [ ]:
df.shape

In [ ]:
pd.cut(df.value, range(0, 105, 10), right=False)

In [ ]:
labels = ["{0} - {1}".format(i, i + 9) for i in range(0, 100, 10)]
labels

In [ ]:
pd.cut(df.value, range(0, 105, 10), right=False, labels=labels)

In [ ]:
df['Group'] = pd.cut(df.value, range(0, 105, 10), right=False, labels=labels)
df

In [ ]:
df.groupby('Group').count()

In [ ]:
df = pd.DataFrame({'value': np.random.randint(0, 100, 20)})
df

In [ ]:
[ df.quantile(q) for q in [.25, .5, .75] ] 

In [ ]:
pd.qcut(df['value'], q=4)

In [ ]:
df['quartile']=pd.qcut(df['value'], q=4)
df

In [ ]:
df['quartile']=pd.qcut(df['value'], q=4, labels=range(1,5))
df

---
# Skalowanie <a id="scale"></a>

- ## Logarytmiczne

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('data/adverts_29_04.csv', sep=';')
data

In [ ]:
data['Price per m2'] = data['Price'] / data['Size (m2)']
data = data.dropna(subset=['Price per m2'])
data['Price per m2'] = data['Price per m2'].map('{:.0f}'.format)
data["day"] = data['Date'].str[0:2]
data["month"] = data['Date'].str[3:5]
data["year"] = data['Date'].str[6:]
df = data.drop(['Price', 'Date'], axis=1)
df

In [ ]:
import pandas as pd
from numpy import log2

data = pd.read_csv('data/adverts_29_04.csv', sep=';')
data['Price per m2'] = data['Price'] / data['Size (m2)']
data['Price per m2'] = data['Price per m2'].map('{:.0f}'.format)
data = data.dropna(subset=['Price per m2'])
df = data.drop(['Price', 'Date'], axis=1)
data["Price log"] = data['Size (m2)'].apply(lambda x: log2(x)).map('{:.2f}'.format)
data["day"] = data['Date'].str[0:2]
data["month"] = data['Date'].str[3:5]
data["year"] = data['Date'].str[6:]
data = data.dropna(subset=['Price per m2'])
df = data.drop(['Price', 'Date'], axis=1)
df

---
- ## Min-Max Scaling

#### Skaluje i przesuwa dane tak, by się mieściły między `0` a `1`

### $$x_{minmax}^i = \frac{x^i-min(x)}{max(x)-min(x)}$$



In [ ]:
df[["Price per m2", "Size (m2)"]]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(df[["Price per m2", "Size (m2)"]])

In [ ]:
scaler.data_max_

In [ ]:
pd.DataFrame(scaler.transform(df[["Price per m2", "Size (m2)"]]))

In [ ]:
pd.DataFrame(scaler.fit_transform(df[["Price per m2", "Size (m2)"]]))


- ## Robust scaling
#### Podobny to skalowania min-max tylko odejmuje medianę i skaluję odległością miedzy 1szym a 3cim kwartylem


In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

pd.DataFrame(scaler.fit_transform(df[["Price per m2", "Size (m2)"]]))


- ## Standaryzacja

#### Standaryzacja polega na sprowadzeniu dowolnego rozkładu normalnego do rozkładu standaryzowanego o wartości oczekiwanej `0` i odchyleniu standardowym `1`.


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

pd.DataFrame(scaler.fit_transform(df[["Price per m2", "Size (m2)"]]))

---

# UWAGA

##  <span style="color: red">NIE</span> zamieniać danych <span style="color: cyan">RZADKICH (dużo zer)</span> w <span style="color: cyan">GĘSTE (mało zer)</span>

---


## Wygładzanie Laplace'a


- ### Dodaj `1` do liczników (zaczynaj od `1` a nie `0`)
- ### Uodparnia model na pomijanie całkowite mało prawdopodobnych zdarzeń
- ### Dobrze radzi sobie z liczeniem __*względnych*__ wartości 

## Przykład: rzut monetą asymetryczną

### $n_0$ - ile razy wypadła  "reszka"
### $n_1$ - ile razy wypadł "orzeł"

### Estymator: 
### $\hat{p} = \frac{n_0+1}{n_0 + n_1 + 2}$
### jest lepszy (mniejszy błąd średnio-kwadratowy) od
### Estymator: $\hat{p} = \frac{n_0}{n_0 + n_1}$


---
# Zmienne kategoryczne <a id="cat"></a>

##  Indeksacja

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

label_encoded = df

label_encoded['Location_Cat'] = labelencoder.fit_transform(label_encoded['Location'])
label_encoded

---
## __*One-hot encoding*__ 

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')

enc_df = pd.DataFrame(enc.fit_transform(label_encoded[['Location_Cat']]).toarray())

one_hot_data = label_encoded.join(enc_df)
one_hot_data

In [ ]:
dum_df = pd.get_dummies(df, columns=['Location'])
dum_df

In [ ]:
dum_df = pd.get_dummies(data, columns=['Location', 'Sold by', 'Type', 'Rooms no.', 'Bathroom no.', 'Parking'])
dum_df

In [ ]:
dum_df.columns

---
## Porządek zmiennych kategorycznych

![Clockface](img/clock.png)

### zmieniamy na współrzędne "wskazówek"

### $ m \to ( \sin{(\frac{2\Pi\:m}{12})}, \cos{(\frac{2\Pi\:m}{12})} )$

In [ ]:
df

In [ ]:
import numpy as np

df['month_x'] = df['month'].apply(lambda x: np.sin(np.pi*int(x)/12))
df['month_y'] = df['month'].apply(lambda x: np.cos(np.pi*int(x)/12))
df